In [289]:
import pandas as pd
import numpy as np
from dstapi import DstApi

import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
import time

In [28]:
ind = DstApi('FOLK1A')

In [66]:
params = {'table':'FOLK1A',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':kom},
                     {'code':'KØN','values':['*']},
                     {'code':'ALDER','values':['*']},
                     {'code':'Tid','values':['*']},
                     {'code':'CIVILSTAND','values':['TOT']}]}
data= ind.get_data(params)

In [265]:
data = pd.read_csv('FOLK1A',index_col=[0])
data['TID'] = pd.to_datetime(data.reset_index()['TID'].apply(change_format))
#data.resample('Y').last()
data = data[data['KØN']=='I alt'][['TID','OMRÅDE','ALDER','INDHOLD']]
data['ÅR'] = data['TID'].dt.year
data = data.groupby(['ÅR','OMRÅDE','ALDER']).last()
data = data.reset_index()
data = data.drop(columns = ['TID'])
data['ÅR'] = data['ÅR'].astype(int)
data = data[data['ALDER'] != 'I alt']
data['ALDER'] = data['ALDER'].str.strip(' år')
data['ALDER'] = data['ALDER'].astype(int)

In [291]:
def plot_graph(year,municipal):
    # Filter the DataFrame based on the selected year
    filtered_df = data[data['ÅR'] == year]
    filtered_df = filtered_df[filtered_df['OMRÅDE']==municipal]

    # Create the bar plot
    plt.barh(filtered_df['ALDER'].unique(),filtered_df['INDHOLD'])
    plt.title(f"Values for year {year}")
    plt.xlabel("Antal")
    plt.ylabel("År")
    plt.show()
    plt.tight_layout()

# Create the year slider
year_slider = widgets.IntSlider(
    value=data['ÅR'].min(),
    min=data['ÅR'].min(),
    max=data['ÅR'].max(),
    step=1,
    description="Year:",
    continuous_update=False,
)

municipal_slider = widgets.Dropdown(
    options=data['OMRÅDE'].unique(),
    value='Ærø',
    description='Municipal:',
    disabled=False,
)


# Create an interactive widget to update the graph based on the year slider
widgets.interactive(plot_graph, year=year_slider, municipal=municipal_slider)


app = TwoByTwoLayout(top_left=year_slider, top_right=municipal_slider,
                     bottom_left=plot_graph, bottom_right=plot_graph)

link_left = jslink((app.top_left, 'value'), (app.bottom_left, 'year'))
link_right = jslink((app.top_right, 'value'), (app.bottom_left, 'municipal'))
app

#for year in range(data['ÅR'].min(), data['ÅR'].max() + 1):
#    year_slider.value = year
#    plot_graph(year)
#    time.sleep(0.2)  
#    clear_output(wait=True)

TraitError: The 'bottom_left' trait of a TwoByTwoLayout instance expected a Widget or None, not the function 'plot_graph'.

In [307]:
data

,ÅR,OMRÅDE,ALDER,INDHOLD
0,2008,Aabenraa,0,610
1,2008,Aabenraa,1,635
2,2008,Aabenraa,10,766
3,2008,Aabenraa,100,2
4,2008,Aabenraa,101,2
...,...,...,...,...
168253,2023,Ærø,95,4
168254,2023,Ærø,96,7
168255,2023,Ærø,97,4
168256,2023,Ærø,98,2


In [325]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

def plot_graph1(change=None):
    plot_graph(output_widget)

def plot_graph2(change=None):
    plot_graph(output_widget2)

def plot_graph(output):
    year = year_slider.value
    municipal = municipal_slider.value
    
    # Filter the DataFrame based on the selected year and municipal
    filtered_df = data[data['ÅR'] == year]
    filtered_df = filtered_df[filtered_df['OMRÅDE'].isin(municipal)]

    # Create the bar plot
    plt.barh(filtered_df['ALDER'].unique(), filtered_df['INDHOLD'])
    plt.title(f"Values for year {year}")
    plt.xlabel("Antal")
    plt.ylabel("År")
    plt.tight_layout()

    # Display the graph in the output widget
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())
        
    plt.clf()
    
def plot_graph_2(output):
    year = year_slider.value
    municipal = municipal_slider.value
    
    xx = data_done.loc[data_done['TID']==year]['GAMMEL_ANDEL']
    yy = data_done.loc[data_done['TID']==year]['SUNDHEDSUDGIFTER']
    
    plt.scatter(xx,yy)
    
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())
        
    plt.clf()

# Create the year slider
year_slider = widgets.IntSlider(
    value=data['ÅR'].min(),
    min=data['ÅR'].min(),
    max=data['ÅR'].max(),
    step=1,
    description="Year:",
    continuous_update=False,
)

# Create the municipal slider
municipal_slider = widgets.SelectMultiple(
    options=data['OMRÅDE'].unique(),
    value=['Ærø'],
    description='Municipal:',
    disabled=False,
)

# Create the output widgets
output_widget = widgets.Output()
output_widget2 = widgets.Output()

# Observe the slider changes and update the graphs
year_slider.observe(plot_graph1, names='value')
municipal_slider.observe(plot_graph1, names='value')

year_slider.observe(plot_graph2, names='value')
municipal_slider.observe(plot_graph2, names='value')

# Create a 2x2 GridBox layout
layout = widgets.GridBox(
    children=[year_slider, municipal_slider, output_widget, output_widget2],
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='50% 50%',
        grid_template_rows='25% 50%',
        grid_gap='10px',
        grid_template_areas='''
            "year_slider municipal_slider"
            "output_widget output_widget2"
        ''')
)

# Initialize the graphs
plot_graph1()
plot_graph2()

# Display the GridBox layout
display(layout)


GridBox(children=(IntSlider(value=2008, continuous_update=False, description='Year:', max=2023, min=2008), Sel…

<Figure size 432x288 with 0 Axes>

In [1]:
from dstapi import DstApi

In [283]:
ind = DstApi('REGK11')

params = {'table':'REGK11',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':kom},
                     {'code':'PRISENHED','values':['INDL']},
                     {'code':'DRANST','values':['1']},
                     {'code':'FUNK1','values':['4']},
                     {'code':'ART','values':['TOT']},
                     {'code':'TID','values':['>2007']}]}
data_2 = ind.get_data(params)

data_2.drop(columns=['PRISENHED', 'DRANST', 'FUNK1', 'ART'],inplace=True)

data_2.rename(columns={'INDHOLD':'SUNDHEDSUDGIFTER'},inplace=True)

KeyError: "['PRISENHED', 'DRANST', 'FUNK1', 'ART'] not found in axis"